In [1]:
import os
import langchain
from dotenv import load_dotenv

In [7]:
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

In [3]:
from langchain.llms.openai import OpenAI

llm = OpenAI(temperature = 0.6)
name = llm("who is the father of punjabi music industry?")
print(name)



The father of Punjabi music industry is the legendary singer-songwriter, actor, and music director, Kuldeep Manak.


In [19]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

prompt_template1 = PromptTemplate(
    input_variables = ['text'],
    template = "who is the father of {text} music industry?"
)

chain1 = LLMChain(llm = llm, prompt=prompt_template1)

In [24]:
prompt_template2 = PromptTemplate(
    input_variables = ['artist_name'],
    template = "who is the date of birth of {artist_name}?"
)

chain2 = LLMChain(llm = llm ,prompt=prompt_template2)


In [25]:
from langchain.chains import LLMChain,SimpleSequentialChain

chain = SimpleSequentialChain(chains = [chain1,chain2])
response = chain.run("pollywood")
print(response)



Kuldeep Manak was born on 18 April 1952 in Jandiala Guru, Amritsar district, Punjab, India.


In [20]:
from langchain.chains import SequentialChain

prompt_template1 = PromptTemplate(
    input_variables = ['text'],
    template = "who is the father of {text} music industry?"
)

chain1 = LLMChain(llm = llm, prompt=prompt_template1, output_key="artist_name")

prompt_template2 = PromptTemplate(
    input_variables = ['artist_name'],
    template = "who is the date of birth of {artist_name}?"
)

chain2 = LLMChain(llm = llm ,prompt=prompt_template2, output_key="dob")

chain = SequentialChain(
    chains = [chain1,chain2],
    input_variables = ['text'],
    output_variables = ['artist_name','dob']
)

chain({'text':'pollywood'})

{'text': 'pollywood',
 'artist_name': '\n\nThe father of Punjabi music industry is often credited to the late Kuldeep Manak.',
 'dob': '\n\nKuldeep Manak was born on August 5, 1952.'}

In [11]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms.openai import OpenAI
import wikipedia

tools = load_tools(["wikipedia","llm-math"], llm=llm)

agent = initialize_agent(
    tools,
    llm,
    agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
#    verbose = True
)

agent.run('what is the age of elon musk in 2023?')

'Elon Musk will be 52 years old in 2023.'

In [8]:
os.environ['SERPAPI_API_KEY'] = os.getenv('SERPAPI_API_KEY')

In [17]:
from langchain.agents import AgentType, initialize_agent, load_tools
tools = load_tools(["serpapi","llm-math"], llm=llm)
agent = initialize_agent(
    tools,
    llm,
    agent = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose = True   
)

agent.run("What will be the age of elon musk after 5 years from 2023?")



> Entering new AgentExecutor chain...
 Elon Musk's age is likely a simple calculation
Action: Calculator
Action Input: 2023 + 5
Observation: Answer: 2028
Thought: I now know the final answer
Final Answer: Elon Musk will be 59 years old in 2028.

> Finished chain.


'Elon Musk will be 59 years old in 2028.'

In [21]:
chain = LLMChain(llm=llm, prompt=prompt_template1)
name = chain.run("pollywood")
name

'\n\nThe father of Punjabi music industry, also known as Pollywood, is the legendary singer-songwriter Gurdas Maan.'

In [22]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory()

chain = LLMChain(llm=llm, prompt=prompt_template1,memory=memory)

name = chain.run("pollywood")
print(name)



Ustad Kuldeep Manak is often considered the father of Punjabi music, or "Pollywood."


In [23]:
name = chain.run("bollywood")

In [24]:
print(chain.memory)

chat_memory=ChatMessageHistory(messages=[HumanMessage(content='pollywood'), AIMessage(content='\n\nUstad Kuldeep Manak is often considered the father of Punjabi music, or "Pollywood."'), HumanMessage(content='bollywood'), AIMessage(content='\n\nThe father of Bollywood music industry is composer Naushad Ali.')])


In [25]:
print(chain.memory.buffer)

Human: pollywood
AI: 

Ustad Kuldeep Manak is often considered the father of Punjabi music, or "Pollywood."
Human: bollywood
AI: 

The father of Bollywood music industry is composer Naushad Ali.


In [30]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=2)

chain3 = ConversationChain(
    llm = OpenAI(temperature=0.6),
    memory = memory
)

In [31]:
chain3.run("what is 5+5?")

' The answer is 10.'

In [28]:
chain3.prompt

PromptTemplate(input_variables=['history', 'input'], template='The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\n\nCurrent conversation:\n{history}\nHuman: {input}\nAI:')

In [32]:
chain3.memory

ConversationBufferWindowMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='what is 5+5?'), AIMessage(content=' The answer is 10.')]), k=2)